In [1]:
import torch
from torch import nn
import onnx
import torch.nn.functional as F
from onnx_tf.backend import prepare
import tensorflow as tf

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(4, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 3)
        self.softmax = nn.Softmax(dim=1) 

    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = self.fc2(X)
        X = self.fc3(X)
        return X

model = Model()

2023-02-28 21:13:16.805218: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 21:13:17.571632: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/slurm/slurm-19.05.5/lib:/usr/local/slurm/slurm-19.05.5/lib/slurm:/usr/local/lib:
2023-02-28 21:13:17.571734: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:

In [2]:
onnx_file = "../../results/Coral_iris_model.onnx"
tf_model_path = "../../results/Coral_iris_model.tf"


shape = (10,4)
X = torch.ones(shape, dtype=torch.float32)

torch.onnx.export(
    model,
    X,
    onnx_file,
    opset_version=12,
    do_constant_folding=True,
)

onnx_model =  onnx.load(onnx_file)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(tf_model_path)

2023-02-28 21:13:19.153408: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-02-28 21:13:19.153479: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (wr0): /proc/driver/nvidia/version does not exist
2023-02-28 21:13:19.154217: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


INFO:tensorflow:Assets written to: ../../results/Coral_iris_model.tf/assets


INFO:tensorflow:Assets written to: ../../results/Coral_iris_model.tf/assets


In [3]:
tflite_model_path = "../../results/Coral_iris_model.tflite"


converter = tf.lite.TFLiteConverter.from_saved_model(tf_model_path)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = X
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.target_spec.supported_types = [tf.int8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8


tflite_model = converter.convert()
with open(tflite_model_path, 'wb') as f: 
    f.write(tflite_model)

RuntimeError: Boolean value of Tensor with more than one value is ambiguous